# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

我们现在可以将存储在文件中的数据读回内存。

In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]

In [3]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。

In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
**需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。**
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
X, Y

(tensor([[ 3.5486e-01,  2.2585e+00, -1.1827e+00, -5.5131e-01, -7.9360e-01,
           1.0321e+00, -1.8008e+00,  7.8140e-01,  3.7105e-01, -9.8850e-01,
          -5.2159e-01, -6.5740e-01,  7.7749e-02, -1.0196e+00, -1.1168e+00,
           1.5408e+00,  6.3659e-01,  7.8005e-02, -7.2309e-01,  9.7826e-01],
         [ 4.2258e-01, -1.6792e+00,  5.7807e-01,  1.2039e+00, -5.1406e-02,
           8.3716e-01,  1.6647e-01,  3.4777e-01, -5.6235e-01,  3.2793e-01,
           5.4545e-02, -2.5604e-01,  7.1732e-01, -1.3734e+00, -2.1747e-01,
           1.3123e-01,  1.2164e+00,  8.7483e-01, -5.1444e-01, -6.6577e-04]]),
 tensor([[-0.1264,  0.0921, -0.3392, -0.2413,  0.1113, -0.2561,  0.3629,  0.5479,
           0.0899, -0.4099],
         [-0.1601, -0.0084, -0.2603, -0.2037,  0.1824, -0.1611,  0.1112,  0.1003,
           0.1286, -0.3214]], grad_fn=<AddmmBackward0>))

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]

In [6]:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.1332,  0.0322, -0.1505,  ..., -0.0882,  0.0452,  0.0668],
                      [-0.1153, -0.1790,  0.0884,  ...,  0.1508,  0.1164, -0.1673],
                      [ 0.0049,  0.1845,  0.0719,  ..., -0.2096,  0.1095,  0.1662],
                      ...,
                      [-0.1599,  0.0469, -0.0083,  ...,  0.0888, -0.0866,  0.1037],
                      [-0.1732,  0.1461,  0.0695,  ..., -0.0616, -0.0981, -0.0066],
                      [-0.2098, -0.0056,  0.0070,  ...,  0.0902,  0.1029,  0.1930]])),
             ('hidden.bias',
              tensor([ 0.0214,  0.1554,  0.0400, -0.2069, -0.0713,  0.0069,  0.1689,  0.1613,
                       0.1658, -0.1383, -0.0152, -0.2150,  0.0255,  0.1440, -0.2120, -0.0483,
                      -0.1886, -0.1127, -0.1309,  0.0746, -0.0862,  0.1152, -0.0483,  0.0302,
                       0.0844, -0.1584, -0.1108, -0.1183,  0.1327, -0.1274,  0.1651,  0.2010,
                      -0.0992,

In [7]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)

In [8]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。

注：eval评估模式下dropout会被关掉

In [9]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
1. 如何同时保存网络架构和参数？需要对架构加上什么限制？